In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def get_bootstrap(data, labels, N):
    np.random.seed(42)

    n_samples = data.shape[0] # размер совпадает с исходной выборкой
    bootstrap = []
    
    for i in range(N):
        
        sample_index = np.random.randint(0, n_samples, size=n_samples)
        b_data = data[sample_index]
        b_labels = labels[sample_index]
        
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

In [3]:
# get_bootstrap(classification_data, classification_labels, 2)

In [26]:
def get_subsample(len_sample):
    # будем сохранять не сами признаки, а их индексы
    sample_indexes = list(range(len_sample))

    len_subsample = int(np.round(np.sqrt(len_sample)))
    
    subsample = np.random.choice(sample_indexes, size=len_subsample, replace=False)

    return subsample

In [27]:
# get_subsample()

In [28]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [29]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        prediction = np.mean(self.labels)
        #classes = {}  # сформируем словарь "класс: количество объектов"
        #for label in self.labels:
        #    if label not in classes:
        #        classes[label] = 0
        #    classes[label] += 1
            
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        #prediction = max(classes, key=classes.get)
        return prediction   

In [30]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [31]:
# Расчет прироста

def gain(left_labels, right_labels, root_gini):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [32]:
# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [62]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf_samples = 5

    root_gini = gini(labels)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    feature_subsample_indices = get_subsample(n_features) # выбираем случайные признаки
    
    for index in feature_subsample_indices:
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf_samples or len(false_data) < min_leaf_samples:
                continue
            
            current_gain = gain(true_labels, false_labels, root_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [51]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    gain, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [52]:
def random_forest(data, labels, n_trees):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels))
        
    return forest

In [53]:
# Функция классификации отдельного объекта

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [54]:
    # функция формирования предсказания по выборке на одном дереве

    def predict(data, tree):

        classes = []
        for obj in data:
            prediction = classify_object(obj, tree)
            classes.append(prediction)
        return classes

In [55]:
# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))
#     print(predictions)

    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
#     print(predictions_per_object)

    # выберем в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(np.mean(obj))
        
    return voted_predictions

In [56]:
# tree_vote(my_forest_3, test_data[:5])

In [57]:
#Среднеквадратичная ошибка
def mserror(y, y_pred):
    return (sum((y - y_pred)**2)) / len(y)

In [58]:
def data_processing(X):
    X_ = X.drop(['Id', 'physics', 'chemistry', 'biology', 'english', 'geography', 'history'], axis=1)
    columns_list = X_.columns.tolist()
    for column in columns_list:
        min_el = min(X_[column])
        max_el = max(X_[column])
        for i in range(X_.shape[0]):
            X_[column][i] = (X_[column][i] - min_el) / (max_el - min_el)
    x_nmp = X_.to_numpy()
    return x_nmp

In [48]:
#Обработка исходных данных
Train = pd.read_csv("train.csv")
columns_list = Train.columns.tolist()
y_name = columns_list.pop()
y = Train[[y_name]]
X = Train[columns_list]

x_nmp = data_processing(X)
y_nmp_ = y.to_numpy()
y_nmp_.transpose()
y_nmp = y_nmp_.transpose()[0]

train_data, test_data, train_labels, test_labels = train_test_split(x_nmp, 
                                                                    y_nmp, 
                                                                    test_size=0.3,
                                                                    random_state=1)

In [63]:
n_trees_array = [5, 15, 20, 25, 50, 60]
for n_trees in n_trees_array:
    my_forest_1 = random_forest(train_data, train_labels, n_trees)
    # Получим ответы для обучающей выборки 
    train_answers = tree_vote(my_forest_1, train_data)
    # И получим ответы для тестовой выборки
    test_answers = tree_vote(my_forest_1, test_data)
    # Точность на обучающей выборке
    train_accuracy = mserror(train_labels, train_answers)
    print(f'Точность случайного леса из {n_trees} деревьев на обучающей выборке: {train_accuracy:.3f}')
    # Точность на тестовой выборке
    test_accuracy = mserror(test_labels, test_answers)
    print(f'Точность случайного леса из {n_trees} деревьев на тестовой выборке: {test_accuracy:.3f}')

Точность случайного леса из 5 деревьев на обучающей выборке: 69.125
Точность случайного леса из 5 деревьев на тестовой выборке: 84.099
Точность случайного леса из 15 деревьев на обучающей выборке: 66.890
Точность случайного леса из 15 деревьев на тестовой выборке: 81.882
Точность случайного леса из 20 деревьев на обучающей выборке: 66.728
Точность случайного леса из 20 деревьев на тестовой выборке: 81.395
Точность случайного леса из 25 деревьев на обучающей выборке: 66.393
Точность случайного леса из 25 деревьев на тестовой выборке: 81.070
Точность случайного леса из 50 деревьев на обучающей выборке: 66.435
Точность случайного леса из 50 деревьев на тестовой выборке: 80.758
Точность случайного леса из 60 деревьев на обучающей выборке: 66.662
Точность случайного леса из 60 деревьев на тестовой выборке: 80.815


In [64]:
n_trees = 50
my_forest_1 = random_forest(x_nmp, y_nmp, n_trees)

In [66]:
Test = pd.read_csv("test.csv")
test_data = data_processing(Test)
test_answers = tree_vote(my_forest_1, test_data)

submit = pd.read_csv('sample_submission.csv')
submit['mean_exam_points'] = test_answers
submit.to_csv('tutors_submit.csv', index=False)
submit

,Id,mean_exam_points
0,10000,55.277536
1,10001,65.095506
2,10002,46.621964
3,10003,87.336801
4,10004,84.248478
...,...,...
9995,19995,43.245417
9996,19996,81.090851
9997,19997,57.483822
9998,19998,65.640926
